Вариант № 5: Проверка устойчивости отношения порядка при изменении данных

Дано множество задач с дедлайнами и зависимостями (например, задача X должна быть выполнена до задачи Y).

1. Постройте отношение строгого порядка.
2. Напишите программу на Python, которая проверяет, сохраняется ли это отношение при случайном удалении 10% задач.
3. Предложите стратегию пересчета порядка при изменении данных.
4. Где  в области ИТ пригодится решение задания данного варианта?

In [147]:
import numpy as np
import random

# Задачи и их зависимости
tasks = [
    "Помыть посуду (12:00)",
    "Сходить в магазин (13:00)", 
    "Приготовить обед (14:00) - после 1,2",
    "Позвонить маме (15:00)",
    "Забрать детей (16:00)",
    "Сделать уроки (17:00) - после 5",
    "Постирать белье (18:00)",
    "Погладить одежду (19:00) - после 7",
    "Принять душ (20:00)",
    "Лечь спать (23:00) - после 8,9"
]

# Зависимости: какая задача от каких зависит
dependencies = {
    3: [1, 2],  # Задача 3 зависит от 1 и 2
    6: [5],      # Задача 6 зависит от 5
    8: [7],      # Задача 8 зависит от 7
    10: [8, 9]   # Задача 10 зависит от 8 и 9
}

# Создаем матрицу: строка - от кого, столбец - кому
n = len(tasks)
matrix = np.zeros((n, n), dtype=int)

# Заполняем матрицу единицами где есть зависимости
for task, deps in dependencies.items():
    for dep in deps:
        matrix[dep-1][task-1] = 1

print("Список задач:")
for i, task in enumerate(tasks):
    print(f"{i+1}. {task}")

print("\nМатрица зависимостей:")
print("(строка → столбец: задача зависит от другой)")
print(matrix)

# Удаляем случайную задачу (~10%)
task_to_remove = random.randint(0, n-1)
removed_task = tasks[task_to_remove]

print(f"\nУдаляем задачу: {removed_task}")

# Удаляем задачу из списка
tasks_after = tasks.copy()
del tasks_after[task_to_remove]

# Создаем новую матрицу без удаленной задачи
matrix_after = np.delete(matrix, task_to_remove, axis=0)
matrix_after = np.delete(matrix_after, task_to_remove, axis=1)

print(f"Осталось задач: {len(tasks_after)}")

# Проверяем свойства строгого порядка
def check_properties(mat):
    size = len(mat)
    results = {
        'антирефлексивность': True,  # Нет петель (задача не зависит от себя)
        'антисимметричность': True,   # Нет взаимных зависимостей
        'транзитивность': True        # Если A→B и B→C, то A→C
    }
    
    # Проверка 1: нет ли задач, которые зависят от себя
    for i in range(size):
        if mat[i][i] == 1:
            results['антирефлексивность'] = False
            break
    
    # Проверка 2: нет ли циклов (A→B и B→A)
    for i in range(size):
        for j in range(size):
            if i != j and mat[i][j] == 1 and mat[j][i] == 1:
                results['антисимметричность'] = False
                break
    
    # Проверка 3: транзитивность
    for i in range(size):
        for j in range(size):
            if mat[i][j] == 1:  # если i→j
                for k in range(size):
                    # если j→k, но i→k отсутствует - нарушение
                    if mat[j][k] == 1 and mat[i][k] == 0 and i != k:
                        results['транзитивность'] = False
                        break
    
    return results

# Проверяем свойства после удаления
properties = check_properties(matrix_after)

print("\nПроверка свойств после удаления:")
print("=" * 40)
for prop, value in properties.items():
    print(f"{prop:20}: {'ДА' if value else 'НЕТ'}")

# Проверяем, остался ли строгий порядок
is_still_order = all(properties.values())

print("=" * 40)
if is_still_order:
    print("✅ Отношение осталось строгим порядком")
else:
    print("❌ Отношение перестало быть строгим порядком")

Список задач:
1. Помыть посуду (12:00)
2. Сходить в магазин (13:00)
3. Приготовить обед (14:00) - после 1,2
4. Позвонить маме (15:00)
5. Забрать детей (16:00)
6. Сделать уроки (17:00) - после 5
7. Постирать белье (18:00)
8. Погладить одежду (19:00) - после 7
9. Принять душ (20:00)
10. Лечь спать (23:00) - после 8,9

Матрица зависимостей:
(строка → столбец: задача зависит от другой)
[[0 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]]

Удаляем задачу: Постирать белье (18:00)
Осталось задач: 9

Проверка свойств после удаления:
антирефлексивность  : ДА
антисимметричность  : ДА
транзитивность      : ДА
✅ Отношение осталось строгим порядком


# Задание 3.
Стратегия простая:

Удалили задачу - убрали её из списка и из всех зависимостей

Перепроверили три правила:

Правило 1: Никакая задача не зависит от самой себя

Правило 2: Не должно быть взаимных зависимостей (если А перед Б, то Б не может быть перед А)

Правило 3: Если А перед Б и Б перед В, то А должно быть перед В

Если все три правила работают - порядок правильный
Если хоть одно правило нарушилось - порядок сломался

# Задание 4.

**Применение в IT: системы сборки программ**

**Контекст:** При сборке программ существует строгая последовательность обработки файлов:
- Исходные коды → Объектные файлы → Исполняемый файл

**Конкретный пример:**
1. `main.c` - исходный код
2. `utils.c` - исходный код  
3. `main.o` - зависит от `main.c`
4. `utils.o` - зависит от `utils.c`
5. `program` - зависит от `main.o` и `utils.o`

**Проблема:** При удалении `utils.c` нарушается цепочка сборки:
- `utils.o` не может быть создан
- `program` не может быть собран

**Решение:** Наша программа автоматически обнаруживает это нарушение, проверяя:
1. **Антирефлексивность** - файлы не зависят от себя
2. **Антисимметричность** - нет циклических зависимостей  
3. **Транзитивность** - сохранение цепочки зависимостей

**Практическое значение:** Такая проверка используется в:
- Системах сборки (Make, CMake)
- Пакетных менеджерах (npm, pip)
- Системах оркестрации (Kubernetes)
- Инструментах управления зависимостями

**Вывод:** Задание развивает навык анализа целостности систем, что критически важно для создания надежного программного обеспечения.